In [ ]:
from __future__ import division
from numpy.random import randn
from pandas import Series
import numpy as np
import pandas as pd
import os
from datetime import datetime
import math
np.set_printoptions(precision=4)
import sys; sys.path.append('book_scripts')
%pwd

In [ ]:
# input CSV数据文件
input_stock_data_path = 'E:/project/pychram/traderesp/base/input-csv/2013-2014-day-stock-history/'

In [ ]:
# ========== 遍历数据文件夹中所有股票文件的文件名，得到股票代码列表stock_code_list
stock_code_list = []
for root, dirs, files in os.walk(input_stock_data_path):# 注意：这里请填写数据文件在您电脑中的路径
    if files:
        for f in files:
            if '.csv' in f:
                stock_code_list.append(f.split('.csv')[0])

In [ ]:
stock_code_list

In [ ]:
len(stock_code_list)

In [ ]:
def write_result_to_csv_file(turtle_list,begtime,endtime,N1,N2,year=False):
    # ========== 将算好的数据输出到csv文件 - 注意：这里请填写输出文件在您电脑中的路径
    # output CSV数据文件
    output = pd.DataFrame(turtle_list)
    output_stock_data_path = 'E:/project/pychram/traderesp/strategy/output-csv/'
    output_stock_data_cvs = output_stock_data_path + 'output-stock-history-close-turtle-' + str(N1) + '-' + str(N2) + '-' + begtime.strftime('%Y%m%d') + '-' + endtime.strftime('%Y%m%d') 
    output_stock_data_cvs += '-year' if year == True else ''
    output_stock_data_cvs += '.csv'
    
    with_index = True if year == True else False
    output.to_csv(output_stock_data_cvs,encoding='gbk', index=with_index)

In [ ]:
def gen_stock_data(N1=20,N2=10):
    # ========== 根据上一步得到的代码列表，遍历所有股票，将这些股票合并到一张表格 all_stock 中
    all_stock = pd.DataFrame()
    i=0
    # 遍历每个股票
    for code in stock_code_list:
        # 测试 5 次跳过
        i+=1
        if i>= 5:
            #break
            print i
            pass

        # 从csv文件中读取该股票数据 
        # 注意：这里请填写数据文件在您电脑中的路径
        stock_data = pd.read_csv(input_stock_data_path + code + '.csv',
                                 parse_dates=[2],encoding='gbk')
        
        # print stock_data.columns
        # 选取需要的字段，去除其他不需要的字段
        # 股票代码,股票名称,交易日期,新浪行业,新浪概念,新浪地域,开盘价,最高价,最低价,收盘价,后复权价,前复权价,涨跌幅,成交量,成交额,换手率,流通市值,总市值,是否涨停,是否跌停,市盈率TTM,市销率TTM,市现率TTM,市净率,MA_5,MA_10,MA_20,MA_30,MA_60,MA金叉死叉,MACD_DIF,MACD_DEA,MACD_MACD,MACD_金叉死叉,KDJ_K,KDJ_D,KDJ_J,KDJ_金叉死叉,布林线中轨,布林线上轨,布林线下轨,psy,psyma,rsi1,rsi2,rsi3
        stock_data = stock_data[[ u'交易日期',u'股票代码',u'最高价',u'最低价', u'收盘价',u'涨跌幅']]
        stock_data.columns = ['date','code','high','low', 'close','change']
        stock_data.sort_values(by='date', ascending=True,inplace=True)
        #
        # 通过rolling_max方法计算最近N1个交易日的最高价
        stock_data['n1_high'] =  pd.rolling_max(stock_data['high'], N1)
        # 对于上市不足N1天的数据，取上市至今的最高价
        stock_data['n1_high'].fillna(value=pd.expanding_max(stock_data['high']), inplace=True)
        #
        # 通过相似的方法计算最近N2个交易日的最低价
        stock_data['n2_low'] =  pd.rolling_min(stock_data['low'], N2)
        stock_data['n2_low'].fillna(value=pd.expanding_min(stock_data['low']), inplace=True)
        #
        # 当当天的【close】> 昨天的【最近N1个交易日的最高点】时，将【收盘发出的信号】设定为1
        buy_index = stock_data[stock_data['close'] > stock_data['n1_high'].shift(1)].index
        stock_data.loc[buy_index, 'turble_signal'] = 1
        # 当当天的【close】< 昨天的【最近N2个交易日的最低点】时，将【收盘发出的信号】设定为 0
        sell_index = stock_data[stock_data['close'] < stock_data['n2_low'].shift(1)].index
        stock_data.loc[sell_index, 'turble_signal'] = 0
        #
        # 计算每天的仓位，当天持有上证指数时，仓位为1，当天不持有上证指数时，仓位为0
        stock_data['today_singal'] = stock_data['turble_signal'].shift(1)
        stock_data['today_singal'].fillna(method='ffill', inplace=True)
        #
        # 当仓位为1时，买入上证指数，当仓位为0时，空仓。计算从开始至今的资金指数
        stock_data['change_action'] = (stock_data['change'] * stock_data['today_singal'] + 1.0).cumprod()
        # 当仓位恒为1时，持股的收益
        stock_data['change_stick'] = (stock_data['change'] + 1.0).cumprod()
        #
        # 去掉 stock 代表市场的前两个字符
        trim_stock_code = stock_data['code'].map(lambda x: x[2:])
        stock_data['code'] = trim_stock_code
        #
        # 将该股票的合并到output中
        all_stock = all_stock.append(stock_data, ignore_index=True)
        
    #
    write_result_to_csv_file(all_stock,datetime(2013,1,1),datetime(2014,12,31),N1,N2)
    return all_stock

In [ ]:
all_stock = gen_stock_data()

In [ ]:
all_stock

In [ ]:
# 重设索引并分组。 时间频率按年转换，并取最后的时间数据
all_stock_year = all_stock.set_index('date')[['code','change_stick', 'change_action']].groupby(['code']).resample('A', how='last')

In [ ]:
all_stock_year

In [ ]:
write_result_to_csv_file(all_stock_year,datetime(2013,1,1),datetime(2014,12,31),20,10,True)